# STEP 0: Install dependencies

In [2]:
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 \
langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.9 MB/s eta 0:00:00


# STEP 1: Imports and secure API key input

In [4]:
import os
import getpass
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [6]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Please enter your Gemini API key: ')

Please enter your Gemini API key: ··········


# STEP 2: Initialize Gemini 1.5 Flash

In [8]:
llm = ChatGoogleGenerativeAI(model='models/gemini-1.5-flash-latest',temperature=0.2)

# STEP 3: Node to ask user for symptom

In [9]:
def get_symptom(state:dict) -> dict:
  symptom = input("Welcome to XYZ hospital, Please enter your symptom: ")
  state['symptom'] = symptom
  return state

# STEP 4: Node to classify the symptom

In [10]:
def classify_symptom(state:dict) -> dict:
  prompt=(
      "You are a helpful Medical Assitant, Classify the symptoms below into one of the categories \n"
      "-General\n- Emergency \n -mental health \n"
      f"Symptom:{state['symptom']} \n"
      "Respond only with one word : General, Emergency, Mental Health"
      "#Example: input: I have fever, Output: General"
  )

  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"LLM classifies the symptom as : {category}") #debug
  state['category'] = category
  return state

# STEP 5: Router logic to route to the correct node


In [11]:
def symptom_router(state:dict) -> dict:
  cat = state['category'].lower()  #General , general , Mental mental
  if "general" in cat:
    return "general"

  elif "mental" in cat:
    return "mental_health"

  elif "emergency" in cat:
    return "emergency"

  else:
    return "general"

# STEP 6: Category-specific response nodes


In [12]:
def general_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}': seems general : directing you to general ward for consulting a doctor"
  return state

def emergency_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}': It is Medical Emergency : seeking immediate help"
  return state

def mental_health_node(state:dict) -> dict:
  state["answer"] = f" '{state['symptom']}': seems like a medical health issue: talk to our counsellor "
  return state

# STEP 7: Build LangGraph


In [18]:
builder = StateGraph(dict)
#define the nodes
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.set_entry_point("get_symptom")
builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify",symptom_router,{
    "general":"general",
    "emergency":"emergency",
    "mental_health":"mental_health"
})

builder.add_edge("general",END)
builder.add_edge("emergency",END)
builder.add_edge("mental_health",END)

# STEP 8: Compile and invoke the graph


In [21]:
graph = builder.compile()
final_state = graph.invoke({"symptom":""}) # Pass an empty dictionary as initial state
print("final output \n")
print(final_state["answer"])

Welcome to XYZ hospital, Please enter your symptom: I am sad as my friend Aamir got failed in exam
LLM classifies the symptom as : Mental Health
final output 

 'I am sad as my friend Aamir got failed in exam': seems like a medical health issue: talk to our counsellor 


In [ ]:
graph = builder.compile()
mermaid_graph=graph.get_graph()